#Topic Modelling su dati con recensioni positive

In [ ]:
import os
import numpy as np
import glob
import warnings
import nltk
import string
import collections
import pandas as pd
from shutil import copyfile

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Caricamento dati di train e test che verranno poi uniti

In [ ]:
copyfile('/content/gdrive/MyDrive/Text Mining/TM_Project/test_pos.csv', 'test_pos.csv')
copyfile('/content/gdrive/MyDrive/Text Mining/TM_Project/train_pos.csv', 'train_pos.csv')

'train_pos.csv'

In [ ]:
test_pos=pd.read_csv('test_pos.csv')
train_pos=pd.read_csv('train_pos.csv')

In [ ]:
pos= train_pos.append(test_pos).reset_index()
pos.rename(columns = {'0' : 'text'}, inplace = True)

In [ ]:
#Rimozione di una colonna inutile ai fini dell'analisi
pos.drop(['index','Unnamed: 0'],axis=1, inplace=True)
pos

,text
0,"['woman', 'borough', 'boy', 'world', 'wa', 'al..."
1,"['problem', 'film', 'thought', 'wa', 'pretty',..."
2,"['really', 'liked', 'movie', 'kurt', 'manages'..."
3,"['movie', 'ha', 'problem', 'presentation', 'ma..."
4,"['sum', 'documentary', 'word', 'next', 'imposs..."
...,...
24995,"['1858', 'tolstoy', 'wrote', 'diary', 'politic..."
24996,"['heard', 'wa', 'abc', 'australian', 'broadcas..."
24997,"['movie', 'funny', 'sad', 'enough', 'think', '..."
24998,"['last', 'dinosaur', 'film', 'meant', 'fun', '..."


Caricamento di alcune funzioni utili per il preprocessing

In [ ]:
!wget -q "https://www.dropbox.com/s/yfjx9nhrky1ifws/preprocessing_functions.py" -O preprocessing_functions.py

In [ ]:
 from preprocessing_functions import remove_punctuation, remove_numbers, remove_stopwords,remove_extra_whitespace

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
pos.text = pos.text.apply(remove_punctuation)
pos.text = pos.text.apply(remove_extra_whitespace)
pos.text = pos.text.apply(remove_numbers)

In [ ]:
# Don't print warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

In [ ]:
pos

,text
0,woman borough boy world wa alone alone boy gif...
1,problem film thought wa pretty good actual lap...
2,really liked movie kurt manages act without sp...
3,movie ha problem presentation may even offensi...
4,sum documentary word next impossible every fib...
...,...
24995,tolstoy wrote diary political compatible arti...
24996,heard wa abc australian broadcasting corporati...
24997,movie funny sad enough think kinda true love o...
24998,last dinosaur film meant fun exciting succeeds...


In [ ]:
!pip install -q pyLDAvis==2.1.2

     |████████████████████████████████| 1.6 MB 5.1 MB/s 


In [ ]:
from __future__ import print_function

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

Topic Modelling con gensim: creare il dizionario e poi implementare il modello LDA

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
texts = list(pos.text.str.split())

In [ ]:
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=7, no_above=0.3)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

* Fit LDA model


In [ ]:
lda_gensim = LdaModel(corpus, num_topics=8)

In [ ]:
# Print the Keyword in the 8 topics
lda_gensim.print_topics()


In [ ]:
#Getting the words from the dictionary
dictionary[84] 

In [ ]:
doc_lda = lda_gensim[corpus]

In [ ]:
pyLDAvis.gensim.prepare(lda_gensim, corpus, dictionary, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4     -108.958252 -208.941483       1        1  20.264573
7      186.487061  -98.103249       2        1  15.614783
0       19.245832  160.858780       3        1  15.121764
1      170.658829   75.445076       4        1  14.563979
5       -1.386411  -35.043739       5        1  12.377210
2     -145.800095  106.944611       6        1   8.229568
3       64.834541 -221.825272       7        1   7.576014
6     -196.906036  -58.990913       8        1   6.252108, topic_info=         Term         Freq        Total Category  logprob  loglift
158      show  9597.000000  9597.000000  Default  30.0000  30.0000
728    horror  3458.000000  3458.000000  Default  29.0000  29.0000
855     funny  4413.000000  4413.000000  Default  28.0000  28.0000
1123   comedy  4373.000000  4373.000000  Default  27.0000  27.0000
1114  western  1407.000000  1407.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
518      work   324.008992  6038.112760   Topic8  -6.2320  -0.1528
126     place   288.037460  2795.831880   Topic8  -6.3496   0.4995
390      made   319.729063  7116.534138   Topic8  -6.2452  -0.3305
661      year   304.973239  7433.387065   Topic8  -6.2925  -0.4213
742      look   294.774674  5181.253597   Topic8  -6.3265  -0.0943

[786 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
15447      5  0.982051    aamir
15447      8  0.011691    aamir
12428      3  0.984185  aardman
5863       6  0.962828     abel
524        1  0.295966   acting
...      ...       ...      ...
1350       3  0.003329   zombie
1350       4  0.006658   zombie
1350       5  0.003329   zombie
1350       7  0.745672   zombie
7009       4  0.981078        ’

[2685 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 1, 2, 6, 3, 4, 7])

Evaluation metrics

In [ ]:
print('Perplexity: ',
lda_gensim.log_perplexity(corpus))

Perplexity:  -8.393484375694323


In [ ]:
from gensim.models import CoherenceModel

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_gensim, texts=texts, dictionary=dictionary, coherence='c_v')

In [ ]:
coherence_lda = coherence_model_lda.get_coherence()    
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.2938597729134953


In [ ]:
lda_5_topic = LdaModel(corpus, num_topics=5)
coherence_model_lda_5_topic = CoherenceModel(model=lda_5_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_5_topic = coherence_model_lda_5_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_5_topic)

Coherence Score:  0.28237074193986667


In [ ]:
lda_10_topic = LdaModel(corpus, num_topics=10)
coherence_model_lda_10_topic = CoherenceModel(model=lda_10_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_10_topic = coherence_model_lda_10_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_10_topic)

Coherence Score:  0.2905947703325832


In [ ]:
lda_15_topic = LdaModel(corpus, num_topics=15)
coherence_model_lda_15_topic = CoherenceModel(model=lda_15_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_15_topic = coherence_model_lda_15_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_15_topic)

Coherence Score:  0.29483812483059907


In [ ]:
lda_19_topic = LdaModel(corpus, num_topics=19)
coherence_model_lda_19_topic = CoherenceModel(model=lda_19_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_19_topic = coherence_model_lda_19_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_19_topic)

Coherence Score:  0.30318468462129494


In [ ]:
#pyLDAvis.gensim.prepare(lda_8_topic, corpus, dictionary, mds='tsne')

In [ ]:
#pyLDAvis.gensim.prepare(lda_5_topic, corpus, dictionary, mds='tsne')